
```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 
                                                                                        

# Demo 4 - Files to MSSQL

## Import severals CSV files to one MSSQL Table

FastTransfer use an embedded DuckDB that allow to read CSV, JSON, Parquet files using SQL to read data.
DuckDB infer the source CSVs structure using the first 100_000 rows if no format is provided.
It is possible to give or force a format. Use the [DuckDB CSV import documentation](https://duckdb.org/docs/stable/data/csv/overview) to see what is possible


### using Datadriven and filenames as distributedkeycolumn

DuckDB allow to retrieve data from CSV as a sql query that FastTransfer will understand and push to a target (MSSQL but also PostgreSQL, Oracle, MySQL, SAP HANA, Clickhouse). This allow **FastTransfer to become an universal CSV parallel loader**. For that you must use sourceconnectiontype as **duckdbstream**

Adding this to the automapping method using column names and you have the perfect import tool !

In [5]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceserver ":memory:" `
--query "SELECT * exclude filename FROM read_csv('D:\temp\mssql\*orders*.csv', filename=true)" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
--method "DataDriven" `
--distributeKeyColumn "filename" `
--datadrivenquery "select file from glob('D:\temp\mssql\*orders*.csv')" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 16 `
--runid "CSV_To_MSSQL_Parallel16_DataDriven_filename"

2025-09-01T22:43:35.133 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑09‑19 (17 day(s) left).
2025-09-01T22:43:35.143 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-01T22:43:35.143 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-01T22:43:35.143 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-01T22:43:35.143 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 46232
2025-09-01T22:43:

#### You can also import in parallel one big CSV file. It will be slower for sure but faster than a single threaded import.

In [8]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceserver ":memory:" `
--query "SELECT * FROM read_csv('D:\temp\PGCOPY_ONEBIG\PGCOPY_ORDERS.csv')" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
--method "Ntile" `
--distributeKeyColumn "o_orderkey" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 16 `
--runid "CSV_To_MSSQL_Parallel16_Ntile_o_orderkey"

2025-06-15T22:12:34.619 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_Ntile_o_orderkey -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑07‑14 (28 day(s) left).
2025-06-15T22:12:34.629 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_Ntile_o_orderkey -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-06-15T22:12:34.629 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_Ntile_o_orderkey -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.6.0 Architecture : X64 - Framework : .NET 8.0.16
2025-06-15T22:12:34.629 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_Ntile_o_orderkey -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-06-15T22:12:34.629 +02:00 -|- FastTransfer -|- CSV_To_MSSQL_Parallel16_Ntile_o_orderkey -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 46604
2025-06-15T22:12:34.629 +02:00 -

## Import severals Parquet files to one MSSQL Table

Parquet is a great file format and instead of CSV, the structure of the table inside the parquet file is embedded in the parquet file(s) itself. FastTransfer will use DuckDB under the hood to read parquet data using SQL and will push the data to the target table.

In [6]:
cd D:\FastTransfer\

.\FastTransfer.exe `
--sourceconnectiontype "duckdbstream" `
--sourceserver ":memory:" `
--query "SELECT * exclude filename FROM read_parquet('D:\tpchgen-cli\sf100\*\orders*.parquet', filename=true)" `
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders" `
--method "DataDriven" `
--distributeKeyColumn "filename" `
--datadrivenquery "select file from glob('D:\tpchgen-cli\sf100\*\orders*.parquet')" `
--loadmode "Truncate" `
--mapmethod "Name" `
--degree 12 `
--runid "PARQUET_To_MSSQL_Parallel12_DataDriven_filename"

2025-09-01T22:47:45.512 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer - running in trial mode – will expires on 2025‑09‑19 (17 day(s) left).
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- Starting
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- PARQUET_To_MSSQL_Parallel12_DataDriven_filename -|- INFORMATION -|- tpch_test.dbo.orders -|- Process ID : 38492
2025-09-01T22:47:45.524 +02:00 -|- FastTransfer -|- 

This is generally the fastest import format. To Generate parquet file from databases you can try [FastBCP](https://www.arpe.io/fastbcp/)